In [1]:
import numpy as np 
import bnlearn as bn 
import pandas as pd

In [2]:
from support_graph import *

In [3]:
maga_nx = nx.read_gml('../models/complete_data_best_graph_fold_2.gml')

In [4]:
new_names = {
    "sexo_M": "Sex",
    "grupo_idade": "Age",
    "teve_ec": "CAD",
    "imc": "BMI",
    "_64": "Question_64",
    "_66": "Question_66",
    "_58": "Question_58",
    "_59": "Question_59",
    "_76": "Question_76",
    "_77": "Question_77",
}

# rename all nodes that start with cod, replacing it with ICD-10th
for node in maga_nx.nodes():
    if node.startswith("cod_"):
        new_node = "ICD " + node[4:]  # Replace 'cod' with 'ICD-10th_'
        maga_nx = nx.relabel_nodes(maga_nx, {node: new_node})

maga_nx = nx.relabel_nodes(maga_nx, new_names)


In [5]:
support_graph = create_support_graph(maga_nx, 'CAD')

In [6]:
len(support_graph.nodes)

876489

In [10]:
data = pd.read_csv('/home/joao/Desktop/UFMG/PhD/code/explaining-BN/data/data_ec_filtered.csv')
data = data.rename(columns=new_names)
data.columns = data.columns.str.replace("cod_", "ICD ", regex=True)

In [12]:
data.BMI = pd.qcut(data.BMI, q=10, duplicates='drop')
data.Age = pd.qcut(data.Age, q=10, duplicates='drop')

In [11]:
data.head()

,Age,BMI,CAD,Sex,ICD B57,ICD E03,ICD E10,ICD E11,ICD E14,ICD E66,...,ICD R07,ICD R55,ICD Z13,ICD Z95,Question_66,Question_64,Question_58,Question_59,Question_76,Question_77
0,1930,0.000000,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,2000,0.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2000,19.531250,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,2000,25.559329,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1960,27.885187,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1


In [18]:
evidence_variables = set(['BMI', 'Sex', 'Question_64', 'ICD E66', 'ICD J44', 'ICD E66','ICD E10','ICD B57','ICD E78'])

In [19]:
pruned_sg = prune_support_graph(support_graph, evidence_variables)

In [21]:
len(support_graph.edges)

1023831

In [22]:
len(pruned_sg.edges)

33178

In [20]:
print("\nEdges (Supporter -> Supported):")
for edge in pruned_sg.edges(data=True):
    supporter_label = pruned_sg.nodes[edge[0]]['label']
    supported_label = pruned_sg.nodes[edge[1]]['label']
    print(f"- {supporter_label} -> {supported_label}")


Edges (Supporter -> Supported):
- Age -> CAD
- ICD I10 -> Age
- ICD E11 -> ICD I10
- Question_64 -> ICD E11
- ICD E10 -> ICD E11
- ICD B57 -> ICD I10
- ICD E78 -> ICD I10
- ICD I34 -> ICD I10
- ICD Z95 -> ICD I34
- ICD I49 -> ICD Z95
- ICD R00 -> ICD I49
- ICD G47 -> ICD R00
- ICD E66 -> ICD G47
- ICD E78 -> ICD G47
- ICD I11 -> ICD G47
- Question_59 -> ICD I11
- Question_77 -> Question_59
- Question_64 -> Question_77
- Question_76 -> Question_77
- Question_76 -> Question_58
- Question_64 -> Question_76
- Question_64 -> Question_58
- BMI -> Question_77
- Question_58 -> Question_77
- Question_64 -> Question_58
- Question_76 -> Question_59
- Question_77 -> Question_76
- Question_64 -> Question_77
- BMI -> Question_77
- Question_58 -> Question_77
- Question_64 -> Question_76
- Question_58 -> Question_76
- Question_77 -> Question_58
- Question_64 -> Question_77
- BMI -> Question_77
- Question_64 -> Question_58
- BMI -> ICD G47
- Question_77 -> ICD I49
- Question_59 -> Question_77
- ICD I1